---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [ ]:
def blight_model():
    
    import pandas as pd
    import numpy as np
    import re
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.ensemble import GradientBoostingClassifier
    
    
    train = pd.read_csv('train.csv',encoding = "ISO-8859-1" )
    test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    addresses = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    
    locations = addresses.merge(latlons, how ='outer', left_on = 'address', right_on = 'address').drop('address', axis = 1)
    
    train = train.merge(locations, how = 'inner', left_on = 'ticket_id', right_on = 'ticket_id')
    test = test.merge(locations, how = 'inner', left_on = 'ticket_id', right_on = 'ticket_id')
    
    ticket_id = test['ticket_id']
    
    train['ticket_issued_date'] = pd.to_datetime(train.ticket_issued_date)
    train['hearing_date'] = pd.to_datetime(train.hearing_date)
    test['ticket_issued_date'] = pd.to_datetime(test.ticket_issued_date)
    test['hearing_date'] = pd.to_datetime(test.hearing_date)
    train['Days_difference'] = (train['hearing_date'] - train['ticket_issued_date'])/np.timedelta64(1, 'D')
    test['Days_difference'] = (test['hearing_date'] - test['ticket_issued_date'])/np.timedelta64(1, 'D')
    
    train['mailing_address_str_number'] = train['mailing_address_str_number'].astype('str').apply(lambda x: re.sub("[^0-9]", '0', x))
    test['mailing_address_str_number'] = test['mailing_address_str_number'].astype('str').apply(lambda x: re.sub("[^0-9]", '0', x))
    
    train = train.drop(train.columns.difference([ 'compliance', 'lat', 'lon', 'Days_difference',
                                             'late_fee','mailing_address_str_number']), axis =1).dropna()
    
    test = test.drop(test.columns.difference([ 'lat', 'lon', 'Days_difference','late_fee', 
                                              'mailing_address_str_number']), axis =1) 
    
    test['lat'].fillna(0, axis = 0, inplace = True) 
    test['lon'].fillna(0, axis = 0, inplace = True) 
    test['Days_difference'].fillna(0, axis = 0, inplace = True)
    
    X_tr = train.loc[:,train.columns != 'compliance']
    y_tr = train.loc[:,'compliance'].to_frame().values.reshape(train.shape[0],)
    
    scaler = MinMaxScaler()
    X_tr_scaled = scaler.fit_transform(X_tr)
    test_scaled = scaler.transform(test)
    
    gbc = GradientBoostingClassifier(random_state = 0).fit(X_tr_scaled,y_tr)
    
    
    proba = gbc.predict_proba(test_scaled)
    
    #gbc.classes_
    
    proba = [x[1] for x in proba]
    
    compliance = pd.Series(proba, index = ticket_id, dtype = 'float32').rename('compliance')

    return compliance

blight_model()



import numpy as np
bm = blight_model()
res = '{:40s}'.format('Object Type:')
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += '{:40s}'.format('Data Shape:')
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += '{:40s}'.format('Data Values Type:')
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += '{:40s}'.format('Data Values Infinity:')
res += ['Failed: values should not be infinity\n','Passed\n'][not any(np.isinf(bm))]
res += '{:40s}'.format('Data Values NaN:')
res += ['Failed: values should not be NaN\n','Passed\n'][not any(np.isnan(bm))]
res += '{:40s}'.format('Data Values in [0,1] Range:')
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += '{:40s}'.format('Data Values not all 0 or 1:')
res += ['Failed: values should be scores not predicted labels\n','Passed\n'][not all((bm.isin({0,1,0.0,1.0})))]
res += '{:40s}'.format('Index Type:')
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += '{:40s}'.format('Index Values:')
if bm.index.shape==(61001,):
    res +=['Failed: index values should match test.csv\n','Passed\n'
          ][all(pd.read_csv('test.csv',usecols=[0],index_col=0
                           ).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed: bm.index length should be 61001'
res += '{:40s}'.format('Can run model twice:')
bm2 = None
try:
    bm2 = blight_model()
    res += 'Passed\n'
except:
    res += ['Failed: second run of blight_model() threw an Exception']
res += '{:40s}'.format('Can run model twice with same results:')
if not bm2 is None:
    res += ['Failed: second run of blight_model() produced different results (this might not be a problem)\n','Passed\n'][
        all(bm.apply(lambda x:round(x,3))==bm2.apply(lambda x:round(x,3))) and all(bm.index==bm2.index)]    
print(res)

import pandas as pd
import numpy as np
import re

train = pd.read_csv('train.csv',encoding = "ISO-8859-1" )
test = pd.read_csv('test.csv', encoding = "ISO-8859-1")
addresses = pd.read_csv('readonly/addresses.csv')
latlons = pd.read_csv('readonly/latlons.csv')
    
locations = addresses.merge(latlons, how ='outer', left_on = 'address', right_on = 'address').drop('address', axis = 1)
    
train = train.merge(locations, how = 'inner', left_on = 'ticket_id', right_on = 'ticket_id')
test = test.merge(locations, how = 'inner', left_on = 'ticket_id', right_on = 'ticket_id')

ticket_id = test['ticket_id']

In [ ]:
#Preprocessing

#test['city'] = test['city'].str.capitalize() 
#train['city'] = train['city'].str.capitalize()

#train['city'] = train['city'].replace('Det', 'Detroit') 
#test['city'] = test['city'].replace('Det', 'Detroit')


#train["new_city"] = train['city'].str.replace('[^\w\s]',' ')
#test["new_city"] = test['city'].str.replace('[^\w\s]',' ')


#train['new_city'] = train['new_city'].astype('str').apply(lambda x: re.sub(r'[0-9]+', ' ', x)) 
#test['new_city'] = test['new_city'].astype('str').apply(lambda x: re.sub(r'[0-9]+', ' ', x))


#train['mailing_address_str_number'] = train['mailing_address_str_number'].astype('str').apply(lambda x: re.sub("[^0-9]", '0', x))
#test['mailing_address_str_number'] = test['mailing_address_str_number'].astype('str').apply(lambda x: re.sub("[^0-9]", '0', x))

#train['zip_code'] = train['zip_code'].astype('str').apply(lambda x: re.sub("[^0-9]", '0', x))
#test['zip_code'] = test['zip_code'].astype('str').apply(lambda x: re.sub("[^0-9]", '0', x))

#train['violation_code'] = train['violation_code'].apply(lambda x: re.sub('[^0-9]','', x))
#test['violation_code'] = test['violation_code'].apply(lambda x: re.sub('[^0-9]','', x))

#city = np.concatenate((train['new_city'].unique(),test['new_city'].unique()))
#city = np.unique(city.astype('str'))
#labels_city = dict(zip(city,np.arange(len(city))))
#train['new_city'] = train['new_city'].map(labels_city)
#test['new_city'] = test['new_city'].map(labels_city) 

#state = np.concatenate((train['state'].unique(),test['state'].unique()))
#state = np.unique(state.astype('str'))
#labels_state = dict(zip(state,np.arange(len(state))))
#train['state'] = train['state'].map(labels_state)
#test['state'] = test['state'].map(labels_state)

#agency = np.concatenate((train['agency_name'].unique(),test['agency_name'].unique()))
#agency = np.unique(agency.astype('str'))
#labels_agency = dict(zip(agency,np.arange(len(agency))))
#test['agency_name'] = test['agency_name'].map(labels_agency) 
#train['agency_name'] = train['agency_name'].map(labels_agency) 

#disposition = np.concatenate((train['disposition'].unique(),test['disposition'].unique()))
#disposition = np.unique(disposition.astype('str'))
#labels_disposition = dict(zip(disposition,np.arange(len(disposition))))
#train['disposition'] = train['disposition'].map(labels_disposition) 
#test['disposition']= test['disposition'].map(labels_disposition)

#Preprocessing

train['ticket_issued_date'] = pd.to_datetime(train.ticket_issued_date)
train['hearing_date'] = pd.to_datetime(train.hearing_date)
test['ticket_issued_date'] = pd.to_datetime(test.ticket_issued_date)
test['hearing_date'] = pd.to_datetime(test.hearing_date)
train['Days_difference'] = (train['hearing_date'] - train['ticket_issued_date'])/np.timedelta64(1, 'D')
test['Days_difference'] = (test['hearing_date'] - test['ticket_issued_date'])/np.timedelta64(1, 'D')

train['mailing_address_str_number'] = train['mailing_address_str_number'].astype('str').apply(lambda x: re.sub("[^0-9]", '0', x))
test['mailing_address_str_number'] = test['mailing_address_str_number'].astype('str').apply(lambda x: re.sub("[^0-9]", '0', x))

train = train.drop(train.columns.difference([ 'compliance', 'lat', 'lon', 'Days_difference',
                                             'late_fee','mailing_address_str_number']), axis =1).dropna()

test = test.drop(test.columns.difference([ 'lat', 'lon', 'Days_difference','late_fee', 'mailing_address_str_number']), axis =1) 

test['lat'].fillna(0, axis = 0, inplace = True) 
test['lon'].fillna(0, axis = 0, inplace = True) 
test['Days_difference'].fillna(0, axis = 0, inplace = True)



print(train[train['compliance']==0].count()[0])
print(train[train['compliance']==1].count()[0])

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()

sns.pairplot(train.iloc[:,0:6], hue='compliance', diag_kind='scatter')

plt.show()

from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split

X_tr = train.loc[:,train.columns != 'compliance']
y_tr = train.loc[:,'compliance'].to_frame().values.reshape(train.shape[0],)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_tr_scaled = scaler.fit_transform(X_tr)
test_scaled = scaler.transform(test)

X_train, X_test, y_train, y_test = train_test_split(X_tr_scaled, y_tr)
    
rfc = RandomForestClassifier().fit(X_train, y_train)

gbc = GradientBoostingClassifier().fit(X_train,y_train)

tree = DecisionTreeClassifier().fit(X_train,y_train)

dummy = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)


### Training fit scores

print(tree.score(X_train,y_train))
print(rfc.score(X_train,y_train))
print(gbc.score(X_train,y_train))
print(dummy.score(X_train,y_train))

# AUC test score

from sklearn.metrics import roc_auc_score

y_predict_rfc = rfc.predict_proba(X_test)
y_predict_tree = tree.predict_proba(X_test)
y_predict_gbc = gbc.predict_proba(X_test)
y_predict_dummy = dummy.predict_proba(X_test)

y_predict_rfc = [x[1] for x in y_predict_rfc]
y_predict_tree = [x[1] for x in y_predict_tree]
y_predict_gbc = [x[1] for x in y_predict_gbc]
y_predict_dummy = [x[1] for x in y_predict_dummy]

AUC_rfc = roc_auc_score(y_test, y_predict_rfc)
AUC_tree = roc_auc_score(y_test, y_predict_tree)
AUC_gbc = roc_auc_score(y_test, y_predict_gbc)
AUC_dummy = roc_auc_score(y_test, y_predict_dummy)
         
print('AUC_rfc:{}'.format(AUC_rfc))
print('AUC_tree:{}'.format(AUC_tree))
print('AUC_gbc:{}'.format(AUC_gbc))
print('AUC_dummy:{}'.format(AUC_dummy))

# GridSearch

from sklearn.model_selection import GridSearchCV

grid_rfc = [{'n_estimators' : [10,15,20], 'max_depth': [3,4,5]}]
grid_rfc_auc = GridSearchCV(rfc, param_grid = grid_rfc, scoring = 'roc_auc',cv=5)
grid_rfc_auc.fit(X_train, y_train)
rfc_best_params = grid_rfc_auc.best_params_
rfc_best_score = grid_rfc_auc.best_score_
rfc_score = grid_rfc_auc.cv_results_

print('rfc_best_params:{}'.format(rfc_best_params), 'rfc_best_score:{}'.format(rfc_best_score))


# GridSearch

from sklearn.model_selection import GridSearchCV

grid_gbc = [{'max_depth' : [3,4,5], 'learning_rate':[0.05, 0.1,0.2]}]
grid_gbc_auc = GridSearchCV(gbc, param_grid = grid_gbc, scoring = 'roc_auc',cv=5)
grid_gbc_auc.fit(X_train, y_train)
gbc_best_params = grid_gbc_auc.best_params_
gbc_best_score = grid_gbc_auc.best_score_
gbc_score = grid_gbc_auc.cv_results_

print('gbc_best_params:{}'.format(gbc_best_params), 'gbc_best_score:{}'.format(gbc_best_score))


# Feature importance

print('tree Feature importances: {}'.format(dict(zip(np.array(X_tr.columns),tree.feature_importances_))))

print('rfc Feature importances: {}'.format(dict(zip(np.array(X_tr.columns),rfc.feature_importances_))))

print('gbc Feature importances: {}'.format(dict(zip(np.array(X_tr.columns),gbc.feature_importances_))))

# Confusion Matrices

from sklearn.metrics import confusion_matrix

print('Confusion_dummy:{}'.format(confusion_matrix(y_test, y_predict_dummy).reshape(1,4)))

print('Confusion_rfc:{}'.format(confusion_matrix(y_test, y_predict_rfc).reshape(1,4)))

print('Confusion_tree:{}'.format(confusion_matrix(y_test, y_predict_tree).reshape(1,4)))

print('Confusion_gbc:{}'.format(confusion_matrix(y_test, y_predict_gbc).reshape(1,4)))

#print('Confusion_grid:{}'.format(confusion_matrix(y_test, y_predict_grid).reshape(1,4)))


    
train['ticket_issued_date'] = pd.to_datetime(train.ticket_issued_date)
train['hearing_date'] = pd.to_datetime(train.hearing_date)
test['ticket_issued_date'] = pd.to_datetime(test.ticket_issued_date)
test['hearing_date'] = pd.to_datetime(test.hearing_date)
    
train['ticket_issued_month'] = train['ticket_issued_date'].dt.month
train['hearing_month'] = train['hearing_date'].dt.month.astype('float')
test['ticket_issued_month'] = test['ticket_issued_date'].dt.month
test['hearing_month'] = test['hearing_date'].dt.month
    
train = train.drop(['ticket_issued_date','hearing_date' ], axis = 1)
test = test.drop(['ticket_issued_date','hearing_date' ], axis = 1)
    
train = pd.get_dummies(train, prefix = ['city','zip_code','state'])
test = pd.get_dummies(test, prefix = ['city','zip_code','state'])
